In [ ]:
%git clone https://github.com/Hotsnown/seminaire-bordeaux-2022.git seminaire &> /dev/null
%pip install nbautoeval &> /dev/null
from evaluation.jour2.listes.listes import exo_create_list, exo_add_list, exo_lenght, exo_get_item, exo_is_empty, exo_less_than_5, exo_first_last

Training a machine learning model with scikit-learn
From the video series: Introduction to machine learning with scikit-learn

Agenda
What is the K-nearest neighbors classification model?
What are the four steps for model training and prediction in scikit-learn?
How can I apply this pattern to other machine learning models?
Reviewing the iris dataset

In [ ]:
from IPython.display import IFrame
IFrame('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', width=300, height=200)


150 observations
4 features (sepal length, sepal width, petal length, petal width)
Response variable is the iris species
Classification problem since response is categorical
More information in the UCI Machine Learning Repository

K-nearest neighbors (KNN) classification
Pick a value for K.
Search for the K observations in the training data that are "nearest" to the measurements of the unknown iris.
Use the most popular response value from the K nearest neighbors as the predicted response value for the unknown iris.

Loading the data


In [ ]:
# import load_iris function from datasets module
from sklearn.datasets import load_iris

# save "bunch" object containing iris dataset and its attributes
iris = load_iris()

# store feature matrix in "X"
X = iris.data

# store response vector in "y"
y = iris.target

In [ ]:
# print the shapes of X and y
print(X.shape)
print(y.shape)

scikit-learn 4-step modeling pattern

Step 1: Import the class you plan to use

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


Step 2: "Instantiate" the "estimator"

"Estimator" is scikit-learn's term for model
"Instantiate" means "make an instance of"

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)


Name of the object does not matter
Can specify tuning parameters (aka "hyperparameters") during this step
All parameters not specified are set to their defaults

In [ ]:
print(knn)

Step 3: Fit the model with data (aka "model training")

Model is learning the relationship between X and y
Occurs in-place

In [ ]:
knn.fit(X, y)

Step 4: Predict the response for a new observation

New observations are called "out-of-sample" data
Uses the information it learned during the model training process

In [ ]:
knn.predict([[3, 5, 4, 2]])


Returns a NumPy array
Can predict for multiple observations at once

In [ ]:
X_new = [[3, 5, 4, 2], [5, 4, 3, 2]]
knn.predict(X_new)

Using a different value for K


In [ ]:
# instantiate the model (using the value K=5)
knn = KNeighborsClassifier(n_neighbors=5)

# fit the model with data
knn.fit(X, y)

# predict the response for new observations
knn.predict(X_new)

Using a different classification model


In [ ]:
# import the class
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X, y)

# predict the response for new observations
logreg.predict(X_new)

Resources

Nearest Neighbors (user guide), KNeighborsClassifier (class documentation)
Logistic Regression (user guide), LogisticRegression (class documentation)
Videos from An Introduction to Statistical Learning
Classification Problems and K-Nearest Neighbors (Chapter 2)
Introduction to Classification (Chapter 4)
Logistic Regression and Maximum Likelihood (Chapter 4)